In [29]:
import pandas as pd
import regex as re
import spacy

### Reading each dataset

In [2]:
monster_data = pd.read_csv("../data/cleaned/All_MonsterSg.csv", index_col=0)
monster_data.drop(["roles", "function"], axis=1, inplace=True)
monster_data.reset_index(inplace=True)
monster_data["source"] = 'monster'

monster_data.head(1)

,job_title,company,salary,job_type,years_experience,tech_stack,job_description,industry,date_posted,url,source
0,Senior Python Developer,Citi,NaN,permanent,NaN,NaN,Job Description :\r\n\r\nJob Purpose:\r\nWe ar...,['Banking/Accounting/Financial Services'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster


In [3]:
linkedin_data = pd.read_csv("../data/cleaned/LinkedIn.csv", index_col=0)
linkedin_data.drop(["applicantsCount", "remoteAllowed", "job_functions", "companyUrl"], axis=1, inplace=True)
linkedin_data.reset_index(inplace=True)
linkedin_data["source"] = 'linkedin'

linkedin_data.head(1)

,job_title,date_posted,job_desc,company_name,industry,job_type,url,source
0,Cloud Consumption Analyst,2021-10-07,Job Description\r\n\r\nThe Cloud Consumption A...,Intel Corporation,Semiconductors,permanent,https://www.linkedin.com/jobs/view/2773797384/,linkedin


In [4]:
nodeflair_data = pd.read_csv("../data/cleaned/cleaned_nodeflair_jobpostings.csv", index_col=0)
nodeflair_data["source"] = 'nodeflair'

nodeflair_data.head(1)

,url,job_title,company_name,salary,job_type,years_of_experience,tech_stack,job_desc,date_posted,source
0,https://www.nodeflair.com//jobs/53907,ReactJS Developer (Full Stack),Apar Technologies,"['6,419', '8,819']",Permanent,"[2, 3]","['Docker', 'CloudFoundry', 'Spring', 'SonarQub...",We are looking for a candidate to fill in the ...,2022-03-10,nodeflair


In [5]:
print("Monster columns:", monster_data.columns)
print("LinkedIn columns:", linkedin_data.columns)
print("NodeFlair columns:", nodeflair_data.columns)

Monster columns: Index(['job_title', 'company', 'salary', 'job_type', 'years_experience',
       'tech_stack', 'job_description', 'industry', 'date_posted', 'url',
       'source'],
      dtype='object')
LinkedIn columns: Index(['job_title', 'date_posted', 'job_desc', 'company_name', 'industry',
       'job_type', 'url', 'source'],
      dtype='object')
NodeFlair columns: Index(['url', 'job_title', 'company_name', 'salary', 'job_type',
       'years_of_experience', 'tech_stack', 'job_desc', 'date_posted',
       'source'],
      dtype='object')


### Standardising column names

In [6]:
monster_data.rename(columns={'company':'company_name', 'years_experience':'years_of_experience','job_description':'job_desc'}, inplace=True)

monster_data.head(1)

,job_title,company_name,salary,job_type,years_of_experience,tech_stack,job_desc,industry,date_posted,url,source
0,Senior Python Developer,Citi,NaN,permanent,NaN,NaN,Job Description :\r\n\r\nJob Purpose:\r\nWe ar...,['Banking/Accounting/Financial Services'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster


### Combining all 3 datasets

In [10]:
combined_data = pd.concat([monster_data, linkedin_data, nodeflair_data])

In [11]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11448 entries, 0 to 2900
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_title            11448 non-null  object
 1   company_name         11448 non-null  object
 2   salary               5711 non-null   object
 3   job_type             11441 non-null  object
 4   years_of_experience  8969 non-null   object
 5   tech_stack           7976 non-null   object
 6   job_desc             11437 non-null  object
 7   industry             9131 non-null   object
 8   date_posted          10908 non-null  object
 9   url                  11448 non-null  object
 10  source               11448 non-null  object
dtypes: object(11)
memory usage: 1.0+ MB


In [12]:
combined_data.head(1)

,job_title,company_name,salary,job_type,years_of_experience,tech_stack,job_desc,industry,date_posted,url,source
0,Senior Python Developer,Citi,NaN,permanent,NaN,NaN,Job Description :\r\n\r\nJob Purpose:\r\nWe ar...,['Banking/Accounting/Financial Services'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster


### Extracting Salary Limits and Years of Experience Limits to individual columns

In [ ]:
def get_lower_salary(text):
    if text == "NaN":
        return "NaN"

    elif type(text) != float:
        text = text[1:-1].split(", ")

        lower_salary = text[0].replace(",", "")
        return lower_salary[1:-1]

    else:
        return "NaN"

def get_upper_salary(text):
    if text == "NaN":
        return "NaN"

    elif type(text) != float:
        text = text[1:-1].split(", ")

        lower_salary = text[1].replace(",", "")
        return lower_salary[1:-1]

    else:
        return "NaN"

In [ ]:
def get_lower_year(text):
    if text == "NaN":
        return "NaN"

    elif type(text) != float:
        if len(text.split(",")) == 1:
            return "NaN"
        else:
            text = text[1:-1].split(", ")

            lower_year = text[0].replace(",", "")
            return lower_year[1:-1]

    else:
        return "NaN"

def get_upper_year(text):
    if text == "NaN":
        return "NaN"

    elif type(text) != float:
        if len(text.split(",")) == 1:
            return text[1:-1]
        else:
            text = text[1:-1].split(", ")
            # print(text)
            # print(text[1])

            upper_year = text[1].replace(",", "")
            return upper_year[1:-1]

    else:
        return "NaN"

In [24]:
combined_data["lower_salary"] = combined_data["salary"].apply(get_lower_salary)
combined_data["upper_salary"] = combined_data["salary"].apply(get_upper_salary)
combined_data["lower_year"] = combined_data["years_of_experience"].apply(get_lower_year)
combined_data["upper_year"] = combined_data["years_of_experience"].apply(get_upper_year)

combined_data.drop(columns=["salary", "years_of_experience"], inplace=True)

combined_data.head()

,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year
0,Senior Python Developer,Citi,permanent,NaN,Job Description :\r\n\r\nJob Purpose:\r\nWe ar...,['Banking/Accounting/Financial Services'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster,NaN,NaN,NaN,NaN
1,Senior Partner Solutions Architect (Microsoft),Amazon,permanent,NaN,Job Description :\r\nJob summary\r\nDESCRIPTIO...,['Internet/E-commerce'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster,NaN,NaN,NaN,NaN
2,IT Technician,Ascend Com Pte. Ltd.,permanent,"['Switches', 'Mac', 'Windows 10', 'Cloud Compu...",Responsibilities:\r\nProvide helpdesk support ...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,24000,36000,2,5
3,Customer Engineer,Applied Materials South East Asia Pte. Ltd.,permanent,"['Scalability', 'Cloud Computing', 'Google Clo...",Key Responsibilities\r\nPerforms all standard ...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,42000,84000,2,5
4,Customer Engineer,Applied Materials South East Asia Pte. Ltd.,permanent,"['Scalability', 'Cloud Computing', 'Google Clo...",Has developed specialized skills or is multi-s...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,46800,93600,2,5


### Consolidating Tech Stack for each Job Listing

In [25]:
skills_ner_model = spacy.load('../analysis/custom_ner_model')
ner=skills_ner_model.get_pipe('ner')

move_names = list(ner.move_names)
assert skills_ner_model.get_pipe("ner").move_names == move_names

In [26]:
def get_skills(text):
    skills_entities = []

    try:
        for sentence in text.split("."):
            doc = skills_ner_model(sentence)
            for ent in doc.ents:
                skills_entities.append(ent.text)
    except Exception as e:
        return ""

    return set(skills_entities)

In [32]:
def clean_job_description(text):

    text = re.sub(r'\w*\d\w*','', str(text))
    text = text.replace("\r", " ")
    text = text.replace ("\n", " ")
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()

    return text

In [33]:
combined_data["cleaned_job_desc"] = combined_data["job_desc"].apply(clean_job_description)
combined_data["extracted_tech_stack"] = combined_data["cleaned_job_desc"].apply(get_skills)

combined_data.head()

,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year,cleaned_job_desc,extracted_tech_stack
0,Senior Python Developer,Citi,permanent,NaN,Job Description :\r\n\r\nJob Purpose:\r\nWe ar...,['Banking/Accounting/Financial Services'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster,NaN,NaN,NaN,NaN,job description job purpose we are lookin...,"{jasminkarma junit, job description, tableau, ..."
1,Senior Partner Solutions Architect (Microsoft),Amazon,permanent,NaN,Job Description :\r\nJob summary\r\nDESCRIPTIO...,['Internet/E-commerce'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster,NaN,NaN,NaN,NaN,job description job summary description do...,"{microsoft, alwayson }"
2,IT Technician,Ascend Com Pte. Ltd.,permanent,"['Switches', 'Mac', 'Windows 10', 'Cloud Compu...",Responsibilities:\r\nProvide helpdesk support ...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,24000,36000,2,5,responsibilities provide helpdesk support for...,{lan wan}
3,Customer Engineer,Applied Materials South East Asia Pte. Ltd.,permanent,"['Scalability', 'Cloud Computing', 'Google Clo...",Key Responsibilities\r\nPerforms all standard ...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,42000,84000,2,5,key responsibilities performs all standard se...,{daytoday}
4,Customer Engineer,Applied Materials South East Asia Pte. Ltd.,permanent,"['Scalability', 'Cloud Computing', 'Google Clo...",Has developed specialized skills or is multi-s...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,46800,93600,2,5,has developed specialized skills or is multisk...,{}


In [70]:
def consolidate_tech_stack(row):
    all_tech_stack = []

    if row.tech_stack != 'NaN' and type(row.tech_stack) != float:
        text = row.tech_stack[1:-1]
        text = text.replace("'", "")
        text = text.split(",")

        for tech_stack in text:
            all_tech_stack.append(tech_stack)
    
        all_tech_stack += list(row.extracted_tech_stack)

    return set(all_tech_stack)

In [72]:
combined_data["all_tech_stack"] = combined_data.apply(consolidate_tech_stack, axis=1)

combined_data.head()

,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year,cleaned_job_desc,extracted_tech_stack,all_tech_stack
0,Senior Python Developer,Citi,permanent,NaN,Job Description :\r\n\r\nJob Purpose:\r\nWe ar...,['Banking/Accounting/Financial Services'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster,NaN,NaN,NaN,NaN,job description job purpose we are lookin...,"{jasminkarma junit, job description, tableau, ...",{}
1,Senior Partner Solutions Architect (Microsoft),Amazon,permanent,NaN,Job Description :\r\nJob summary\r\nDESCRIPTIO...,['Internet/E-commerce'],2022-02-18,https://www.monster.com.sg/seeker/job-details?...,monster,NaN,NaN,NaN,NaN,job description job summary description do...,"{microsoft, alwayson }",{}
2,IT Technician,Ascend Com Pte. Ltd.,permanent,"['Switches', 'Mac', 'Windows 10', 'Cloud Compu...",Responsibilities:\r\nProvide helpdesk support ...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,24000,36000,2,5,responsibilities provide helpdesk support for...,{lan wan},"{ Windows 7, Routers, Wireless, Predictive ..."
3,Customer Engineer,Applied Materials South East Asia Pte. Ltd.,permanent,"['Scalability', 'Cloud Computing', 'Google Clo...",Key Responsibilities\r\nPerforms all standard ...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,42000,84000,2,5,key responsibilities performs all standard se...,{daytoday},"{ Problem Solving, Scalability, Technical Sal..."
4,Customer Engineer,Applied Materials South East Asia Pte. Ltd.,permanent,"['Scalability', 'Cloud Computing', 'Google Clo...",Has developed specialized skills or is multi-s...,['Other'],2022-02-19,https://www.monster.com.sg/seeker/job-details?...,monster,46800,93600,2,5,has developed specialized skills or is multisk...,{},"{ Problem Solving, Scalability, Technical Sal..."


In [ ]:
combined_data.to_csv("../data/cleaned/combined_data.csv")